# RO System Simulation with Recycle Support Template

This notebook template is used by the simulate_ro_system tool to run WaterTAP simulations.
It supports configurations with recycle streams for high recovery targets.
It accepts parameters via papermill and returns performance results.

In [ ]:
# Parameters cell - will be replaced by papermill
project_root = "/path/to/project"  # Will be replaced by papermill
configuration = {}
feed_salinity_ppm = 5000
feed_temperature_c = 25.0
membrane_type = "brackish"
membrane_properties = None
optimize_pumps = False

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from pyomo.environ import *
from pyomo.network import Arc
import sys
import os

# Add parent directory to path for utils imports using project_root parameter
sys.path.insert(0, project_root)

# Import our elegant initialization utilities
from utils.ro_initialization import (
    calculate_required_pressure,
    initialize_pump_with_pressure,
    initialize_ro_unit_elegant,
    initialize_multistage_ro_elegant
)

# Import membrane properties handler
from utils.membrane_properties_handler import get_membrane_properties

# WaterTAP imports
from watertap.core.solvers import get_solver
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType
)
from watertap.unit_models.pressure_changer import Pump
from watertap.property_models.multicomp_aq_sol_prop_pack import MCASParameterBlock
import watertap.property_models.seawater_prop_pack as props_sw
from watertap.core import ModuleType

# IDAES imports
from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import calculate_scaling_factors
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state
from idaes.core.util import DiagnosticsToolbox
from idaes.models.unit_models import Feed, Product, Mixer, Separator

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(level=logging.INFO)

# Results storage
results = {}

## Check Configuration for Recycle

In [ ]:
# Check if recycle is configured
has_recycle = configuration.get('recycle_ratio', 0) > 0

if has_recycle:
    print(f"Recycle configuration detected:")
    print(f"  Recycle ratio: {configuration['recycle_ratio']*100:.1f}%")
    print(f"  Recycle flow: {configuration.get('recycle_flow_m3h', 0):.1f} m³/h")
    print(f"  Effective feed flow: {configuration.get('effective_feed_flow_m3h', configuration['feed_flow_m3h']):.1f} m³/h")
    print(f"  Effective feed salinity: {configuration.get('effective_feed_salinity_ppm', feed_salinity_ppm):.0f} ppm")
else:
    print("No recycle configuration detected. Building standard RO model.")

## Build WaterTAP Model with Recycle Support

In [ ]:
def build_ro_model_with_recycle(config_data, feed_salinity_ppm, feed_temperature_c, membrane_type, has_recycle):
    """
    Build WaterTAP RO model with optional recycle support.
    """
    # Create concrete model
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)
    
    # Import TransportModel
    from watertap.core.membrane_channel_base import TransportModel
    
    # Property package - using seawater
    m.fs.properties = props_sw.SeawaterParameterBlock()
    
    # Feed conditions
    fresh_feed_flow_m3_s = config_data['feed_flow_m3h'] / 3600  # Convert to m³/s
    feed_mass_frac = feed_salinity_ppm / 1e6  # Convert ppm to mass fraction
    
    # Create fresh feed unit
    m.fs.fresh_feed = Feed(property_package=m.fs.properties)
    
    if has_recycle:
        # Create recycle split from final concentrate
        m.fs.recycle_split = Separator(
            property_package=m.fs.properties,
            outlet_list=["disposal", "recycle"]
        )
        
        # Create mixer for fresh feed and recycle
        m.fs.feed_mixer = Mixer(
            property_package=m.fs.properties,
            inlet_list=["fresh", "recycle"]
        )
        
        # Effective feed (output of mixer) - this is what goes to RO system
        effective_feed = m.fs.feed_mixer.outlet
    else:
        # No recycle - fresh feed goes directly to RO system
        effective_feed = m.fs.fresh_feed.outlet
    
    # Build stages
    n_stages = config_data['stage_count']
    
    # Create all RO stages and pumps
    for i in range(1, n_stages + 1):
        # Create feed pump for stage
        setattr(m.fs, f"pump{i}", Pump(property_package=m.fs.properties))
        
        # Create RO stage with simplified configuration
        setattr(m.fs, f"ro_stage{i}", ReverseOsmosis0D(
            property_package=m.fs.properties,
            has_pressure_change=True,
            concentration_polarization_type=ConcentrationPolarizationType.none,
            mass_transfer_coefficient=MassTransferCoefficient.none,
            pressure_change_type=PressureChangeType.fixed_per_stage,
            transport_model=TransportModel.SD,  # Use SD model
            module_type=ModuleType.spiral_wound  # Add spiral wound module type
        ))
        
        # Create permeate product for each stage
        setattr(m.fs, f"stage_product{i}", Product(property_package=m.fs.properties))
    
    # Create final products
    if has_recycle:
        m.fs.disposal_product = Product(property_package=m.fs.properties)
    else:
        m.fs.concentrate_product = Product(property_package=m.fs.properties)
    
    # Build connectivity
    if has_recycle:
        # Fresh feed to mixer
        m.fs.fresh_to_mixer = Arc(
            source=m.fs.fresh_feed.outlet,
            destination=m.fs.feed_mixer.fresh
        )
        
        # Mixer to first pump
        m.fs.mixer_to_pump1 = Arc(
            source=m.fs.feed_mixer.outlet,
            destination=m.fs.pump1.inlet
        )
    else:
        # Fresh feed directly to first pump
        m.fs.feed_to_pump1 = Arc(
            source=m.fs.fresh_feed.outlet,
            destination=m.fs.pump1.inlet
        )
    
    # Connect first pump to first RO
    m.fs.pump1_to_ro1 = Arc(
        source=m.fs.pump1.outlet,
        destination=m.fs.ro_stage1.inlet
    )
    
    # Connect permeate from first RO to product
    m.fs.ro1_perm_to_prod = Arc(
        source=m.fs.ro_stage1.permeate,
        destination=m.fs.stage_product1.inlet
    )
    
    # Connect stages
    if n_stages > 1:
        for i in range(1, n_stages):
            # Connect concentrate of stage i to pump i+1
            setattr(
                m.fs, f"stage{i}_to_pump{i+1}",
                Arc(
                    source=getattr(m.fs, f"ro_stage{i}").retentate,
                    destination=getattr(m.fs, f"pump{i+1}").inlet
                )
            )
            # Connect pump i+1 to stage i+1
            setattr(
                m.fs, f"pump{i+1}_to_stage{i+1}",
                Arc(
                    source=getattr(m.fs, f"pump{i+1}").outlet,
                    destination=getattr(m.fs, f"ro_stage{i+1}").inlet
                )
            )
            # Connect permeate to product
            setattr(
                m.fs, f"ro{i+1}_perm_to_prod{i+1}",
                Arc(
                    source=getattr(m.fs, f"ro_stage{i+1}").permeate,
                    destination=getattr(m.fs, f"stage_product{i+1}").inlet
                )
            )
    
    # Connect final concentrate
    final_stage = n_stages
    if has_recycle:
        # Final concentrate to recycle split
        m.fs.final_conc_to_split = Arc(
            source=getattr(m.fs, f"ro_stage{final_stage}").retentate,
            destination=m.fs.recycle_split.inlet
        )
        
        # Recycle split outputs
        m.fs.split_to_disposal = Arc(
            source=m.fs.recycle_split.disposal,
            destination=m.fs.disposal_product.inlet
        )
        
        m.fs.split_to_recycle = Arc(
            source=m.fs.recycle_split.recycle,
            destination=m.fs.feed_mixer.recycle
        )
    else:
        # Final concentrate to product
        m.fs.final_conc_arc = Arc(
            source=getattr(m.fs, f"ro_stage{final_stage}").retentate,
            destination=m.fs.concentrate_product.inlet
        )
    
    # Apply arcs to expand the network
    TransformationFactory("network.expand_arcs").apply_to(m)
    
    # Set membrane properties after model structure is built
    # Get membrane properties from handler
    A_w, B_s = get_membrane_properties(membrane_type, membrane_properties)
    
    for i in range(1, n_stages + 1):
        stage_data = config_data['stages'][i-1]
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Set membrane permeability properties
        ro.A_comp.fix(A_w)  # m/s/Pa
        ro.B_comp[0, 'TDS'].fix(B_s)  # m/s
        
        # Fix permeate pressure and pressure drop
        ro.permeate.pressure.fix(101325)  # 1 atm
        ro.deltaP.fix(-0.5e5)  # -0.5 bar pressure drop
        
        # Set membrane area using spiral wound dimensions
        # Get number of vessels and calculate dimensions
        n_vessels = stage_data.get('vessel_count', stage_data.get('n_vessels', 1))
        elements_per_vessel = 7  # Standard configuration
        
        # For spiral wound modules, we need to set length and width
        # WaterTAP calculates area = length × 2 × width (accounting for folding)
        element_length = 1.016  # 40 inches in meters
        total_length = element_length * elements_per_vessel  # Elements in series
        
        # Calculate required width from target area
        target_area = stage_data.get('membrane_area_m2', stage_data.get('area_m2', n_vessels * 37.16 * 7))
        required_width = target_area / (2 * total_length * n_vessels)
        
        # Set the dimensions
        ro.length.fix(total_length)
        ro.width.fix(required_width * n_vessels)  # Total width for all vessels in parallel
    
    # Set fresh feed conditions
    m.fs.fresh_feed.outlet.flow_mass_phase_comp[0, 'Liq', 'H2O'].fix(
        fresh_feed_flow_m3_s * 1000 * (1 - feed_mass_frac)
    )
    m.fs.fresh_feed.outlet.flow_mass_phase_comp[0, 'Liq', 'TDS'].fix(
        fresh_feed_flow_m3_s * 1000 * feed_mass_frac
    )
    m.fs.fresh_feed.outlet.temperature.fix(273.15 + feed_temperature_c)
    m.fs.fresh_feed.outlet.pressure.fix(101325)  # 1 atm
    
    # Set recycle split if applicable
    if has_recycle:
        # Set split fraction based on configuration
        recycle_split_ratio = config_data.get('recycle_split_ratio', 0.5)
        m.fs.recycle_split.split_fraction[0, "recycle"].fix(recycle_split_ratio)
        m.fs.recycle_split.split_fraction[0, "disposal"].fix(1 - recycle_split_ratio)
    
    # Set pump efficiencies
    for i in range(1, n_stages + 1):
        getattr(m.fs, f"pump{i}").efficiency_pump.fix(0.8)
    
    # Set scaling
    m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
    m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
    calculate_scaling_factors(m)
    
    return m

## Initialize and Solve Model

In [ ]:
def initialize_and_solve_with_recycle(m, config_data, has_recycle, optimize_pumps=False):
    """
    Initialize and solve the RO model with recycle support.
    """
    solver = get_solver()
    
    # Check initial degrees of freedom
    print(f"Initial degrees of freedom: {degrees_of_freedom(m)}")
    
    # Initialize fresh feed
    m.fs.fresh_feed.initialize()
    print("Fresh feed initialized")
    
    if has_recycle:
        # Initialize recycle components
        # First, we need to solve without recycle to get initial values
        print("\nInitializing with zero recycle...")
        
        # Temporarily set recycle to zero
        m.fs.recycle_split.split_fraction[0, "recycle"].fix(0)
        m.fs.recycle_split.split_fraction[0, "disposal"].fix(1)
        
        # Propagate state to mixer
        propagate_state(arc=m.fs.fresh_to_mixer)
        
        # Initialize mixer with zero recycle
        m.fs.feed_mixer.initialize()
        print("Feed mixer initialized")
        
        # Propagate to first pump
        propagate_state(arc=m.fs.mixer_to_pump1)
    else:
        # Propagate directly to first pump
        propagate_state(arc=m.fs.feed_to_pump1)
    
    # Initialize RO stages using elegant approach
    try:
        # Initialize stages sequentially
        for i in range(1, config_data['stage_count'] + 1):
            print(f"\nInitializing Stage {i}...")
            
            pump = getattr(m.fs, f"pump{i}")
            ro = getattr(m.fs, f"ro_stage{i}")
            
            # Calculate required pressure for this stage
            if i == 1:
                feed_tds_ppm = config_data.get('effective_feed_salinity_ppm', config_data['feed_salinity_ppm'])
            else:
                # Estimate TDS for later stages
                feed_tds_ppm = feed_tds_ppm * 1.5  # Rough estimate
            
            stage_recovery = config_data['stages'][i-1].get('stage_recovery', 0.5)
            required_pressure = calculate_required_pressure(
                feed_tds_ppm,
                stage_recovery,
                permeate_pressure=101325,
                min_driving_pressure=15e5,
                pressure_drop=0.5e5
            )
            
            # Initialize pump with required pressure
            initialize_pump_with_pressure(pump, required_pressure)
            
            # Propagate to RO
            if i == 1:
                propagate_state(arc=m.fs.pump1_to_ro1)
            else:
                arc_name = f"pump{i}_to_stage{i}"
                propagate_state(arc=getattr(m.fs, arc_name))
            
            # Initialize RO
            ro.initialize()
            
            # Propagate to product
            if i == 1:
                propagate_state(arc=m.fs.ro1_perm_to_prod)
            else:
                arc_name = f"ro{i}_perm_to_prod{i}"
                propagate_state(arc=getattr(m.fs, arc_name))
            
            # Initialize stage product
            getattr(m.fs, f"stage_product{i}").initialize()
            
            # Propagate concentrate to next stage if not last
            if i < config_data['stage_count']:
                arc_name = f"stage{i}_to_pump{i+1}"
                propagate_state(arc=getattr(m.fs, arc_name))
        
        print("\nAll RO stages initialized successfully!")
        
    except Exception as e:
        print(f"\nInitialization error: {str(e)}")
        raise
    
    # Complete recycle initialization if applicable
    if has_recycle:
        print("\nInitializing recycle components...")
        
        # Propagate final concentrate to split
        propagate_state(arc=m.fs.final_conc_to_split)
        
        # Initialize recycle split
        m.fs.recycle_split.initialize()
        
        # Propagate to disposal
        propagate_state(arc=m.fs.split_to_disposal)
        m.fs.disposal_product.initialize()
        
        # Now set actual recycle ratio
        recycle_split_ratio = config_data.get('recycle_split_ratio', 0.5)
        m.fs.recycle_split.split_fraction[0, "recycle"].fix(recycle_split_ratio)
        m.fs.recycle_split.split_fraction[0, "disposal"].fix(1 - recycle_split_ratio)
        
        print(f"Recycle split set to {recycle_split_ratio*100:.1f}%")
    else:
        # Initialize final concentrate product
        propagate_state(arc=m.fs.final_conc_arc)
        m.fs.concentrate_product.initialize()
    
    # Solve the full model
    print("\nSolving complete model...")
    if has_recycle:
        # Solve with recycle - may need iterations
        print("Solving with recycle flow...")
        solver.options['max_iter'] = 200
        solver.options['tol'] = 1e-6
    
    results = solver.solve(m, tee=True)
    
    if results.solver.termination_condition == TerminationCondition.optimal:
        print("\nSolution found!")
        
        # Report key results
        if has_recycle:
            recycle_flow = value(m.fs.recycle_split.recycle.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                               m.fs.recycle_split.recycle.flow_mass_phase_comp[0, 'Liq', 'TDS'])
            disposal_flow = value(m.fs.disposal_product.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                                m.fs.disposal_product.inlet.flow_mass_phase_comp[0, 'Liq', 'TDS'])
            print(f"\nRecycle flow: {recycle_flow*3.6:.1f} m³/h")
            print(f"Disposal flow: {disposal_flow*3.6:.1f} m³/h")
    
    return results

## Extract Results

In [ ]:
def extract_results_with_recycle(m, config_data, has_recycle):
    """
    Extract simulation results from solved model with recycle support.
    """
    results = {
        "status": "success",
        "performance": {},
        "economics": {},
        "stage_results": [],
        "mass_balance": {},
        "recycle_info": {} if has_recycle else None
    }
    
    # Fresh feed flow
    fresh_feed_flow = value(m.fs.fresh_feed.outlet.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                           m.fs.fresh_feed.outlet.flow_mass_phase_comp[0, 'Liq', 'TDS'])
    
    # Total permeate flow
    total_permeate_flow = sum(
        value(getattr(m.fs, f"ro_stage{i}").permeate.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
              getattr(m.fs, f"ro_stage{i}").permeate.flow_mass_phase_comp[0, 'Liq', 'TDS'])
        for i in range(1, config_data['stage_count'] + 1)
    )
    
    # Recovery based on fresh feed
    results["performance"]["total_recovery"] = total_permeate_flow / fresh_feed_flow
    
    # If recycle, also calculate system recovery
    if has_recycle:
        effective_feed_flow = value(m.fs.feed_mixer.outlet.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                                   m.fs.feed_mixer.outlet.flow_mass_phase_comp[0, 'Liq', 'TDS'])
        results["performance"]["system_recovery"] = total_permeate_flow / effective_feed_flow
        
        # Recycle information
        recycle_flow = value(m.fs.recycle_split.recycle.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                            m.fs.recycle_split.recycle.flow_mass_phase_comp[0, 'Liq', 'TDS'])
        disposal_flow = value(m.fs.disposal_product.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                             m.fs.disposal_product.inlet.flow_mass_phase_comp[0, 'Liq', 'TDS'])
        
        results["recycle_info"] = {
            "recycle_flow_m3h": recycle_flow * 3.6,
            "disposal_flow_m3h": disposal_flow * 3.6,
            "recycle_ratio": recycle_flow / (fresh_feed_flow + recycle_flow),
            "effective_feed_flow_m3h": effective_feed_flow * 3.6,
            "effective_feed_tds_ppm": value(
                m.fs.feed_mixer.outlet.flow_mass_phase_comp[0, 'Liq', 'TDS'] /
                (m.fs.feed_mixer.outlet.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                 m.fs.feed_mixer.outlet.flow_mass_phase_comp[0, 'Liq', 'TDS'])
            ) * 1e6
        }
    
    # Energy consumption
    total_power_kw = sum(
        value(getattr(m.fs, f"pump{i}").work_mechanical[0]) / 1000  # Convert W to kW
        for i in range(1, config_data['stage_count'] + 1)
    )
    
    results["economics"]["total_power_kw"] = total_power_kw
    results["economics"]["specific_energy_kwh_m3"] = (
        total_power_kw / (total_permeate_flow * 3.6)  # m³/s to m³/h
    )
    
    # Stage results
    for i in range(1, config_data['stage_count'] + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        pump = getattr(m.fs, f"pump{i}")
        
        stage_result = {
            "stage_number": i,
            "feed_pressure_bar": value(ro.inlet.pressure[0]) / 1e5,
            "permeate_flow_m3h": value(
                ro.permeate.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                ro.permeate.flow_mass_phase_comp[0, 'Liq', 'TDS']
            ) * 3.6,  # kg/s to m³/h
            "permeate_tds_ppm": value(
                ro.permeate.flow_mass_phase_comp[0, 'Liq', 'TDS'] /
                (ro.permeate.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                 ro.permeate.flow_mass_phase_comp[0, 'Liq', 'TDS'])
            ) * 1e6,
            "concentrate_flow_m3h": value(
                ro.retentate.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                ro.retentate.flow_mass_phase_comp[0, 'Liq', 'TDS']
            ) * 3.6,
            "pump_power_kw": value(pump.work_mechanical[0]) / 1000,
            "water_recovery": value(ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'])
        }
        results["stage_results"].append(stage_result)
    
    # Mass balance verification
    results["mass_balance"]["fresh_feed_flow_m3h"] = fresh_feed_flow * 3.6
    results["mass_balance"]["total_permeate_m3h"] = total_permeate_flow * 3.6
    
    if has_recycle:
        results["mass_balance"]["disposal_flow_m3h"] = disposal_flow * 3.6
        results["mass_balance"]["recycle_flow_m3h"] = recycle_flow * 3.6
        # Balance: fresh feed = permeate + disposal
        balance_error = abs(fresh_feed_flow - total_permeate_flow - disposal_flow)
    else:
        final_stage = config_data['stage_count']
        final_ro = getattr(m.fs, f"ro_stage{final_stage}")
        final_conc_flow = value(
            final_ro.retentate.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
            final_ro.retentate.flow_mass_phase_comp[0, 'Liq', 'TDS']
        )
        results["mass_balance"]["final_concentrate_m3h"] = final_conc_flow * 3.6
        # Balance: feed = permeate + concentrate
        balance_error = abs(fresh_feed_flow - total_permeate_flow - final_conc_flow)
    
    results["mass_balance"]["error_kg_s"] = balance_error
    results["mass_balance"]["error_percent"] = (balance_error / fresh_feed_flow) * 100
    
    return results

def initialize_and_solve_with_recycle(m, config_data, has_recycle, optimize_pumps=False):
    """
    Initialize and solve the RO model with recycle support.
    """
    solver = get_solver()
    
    # Check initial degrees of freedom
    print(f"Initial degrees of freedom: {degrees_of_freedom(m)}")
    
    # Initialize fresh feed
    m.fs.fresh_feed.initialize()
    print("Fresh feed initialized")
    
    if has_recycle:
        # Initialize recycle components
        # First, we need to solve without recycle to get initial values
        print("\nInitializing with zero recycle...")
        
        # Temporarily set recycle to zero
        m.fs.recycle_split.split_fraction[0, "recycle"].fix(0)
        m.fs.recycle_split.split_fraction[0, "disposal"].fix(1)
        
        # Propagate state to mixer
        propagate_state(arc=m.fs.fresh_to_mixer)
        
        # Initialize mixer with zero recycle
        m.fs.feed_mixer.initialize()
        print("Feed mixer initialized")
        
        # Propagate to first pump
        propagate_state(arc=m.fs.mixer_to_pump1)
    else:
        # Propagate directly to first pump
        propagate_state(arc=m.fs.feed_to_pump1)
    
    # Initialize RO stages using elegant approach
    try:
        # Initialize stages sequentially
        for i in range(1, config_data['stage_count'] + 1):
            print(f"\nInitializing Stage {i}...")
            
            pump = getattr(m.fs, f"pump{i}")
            ro = getattr(m.fs, f"ro_stage{i}")
            
            # Calculate required pressure for this stage
            if i == 1:
                # Use effective feed salinity if available, otherwise use feed_salinity_ppm parameter
                feed_tds_ppm = config_data.get('effective_feed_salinity_ppm', feed_salinity_ppm)
            else:
                # Estimate TDS for later stages
                feed_tds_ppm = feed_tds_ppm * 1.5  # Rough estimate
            
            stage_recovery = config_data['stages'][i-1].get('stage_recovery', 0.5)
            required_pressure = calculate_required_pressure(
                feed_tds_ppm,
                stage_recovery,
                permeate_pressure=101325,
                min_driving_pressure=15e5,
                pressure_drop=0.5e5
            )
            
            # Initialize pump with required pressure
            initialize_pump_with_pressure(pump, required_pressure)
            
            # Propagate to RO
            if i == 1:
                propagate_state(arc=m.fs.pump1_to_ro1)
            else:
                arc_name = f"pump{i}_to_stage{i}"
                propagate_state(arc=getattr(m.fs, arc_name))
            
            # Initialize RO
            ro.initialize()
            
            # Propagate to product
            if i == 1:
                propagate_state(arc=m.fs.ro1_perm_to_prod)
            else:
                arc_name = f"ro{i}_perm_to_prod{i}"
                propagate_state(arc=getattr(m.fs, arc_name))
            
            # Initialize stage product
            getattr(m.fs, f"stage_product{i}").initialize()
            
            # Propagate concentrate to next stage if not last
            if i < config_data['stage_count']:
                arc_name = f"stage{i}_to_pump{i+1}"
                propagate_state(arc=getattr(m.fs, arc_name))
        
        print("\nAll RO stages initialized successfully!")
        
    except Exception as e:
        print(f"\nInitialization error: {str(e)}")
        raise
    
    # Complete recycle initialization if applicable
    if has_recycle:
        print("\nInitializing recycle components...")
        
        # Propagate final concentrate to split
        propagate_state(arc=m.fs.final_conc_to_split)
        
        # Initialize recycle split
        m.fs.recycle_split.initialize()
        
        # Propagate to disposal
        propagate_state(arc=m.fs.split_to_disposal)
        m.fs.disposal_product.initialize()
        
        # Now set actual recycle ratio
        recycle_split_ratio = config_data.get('recycle_split_ratio', 0.5)
        m.fs.recycle_split.split_fraction[0, "recycle"].fix(recycle_split_ratio)
        m.fs.recycle_split.split_fraction[0, "disposal"].fix(1 - recycle_split_ratio)
        
        print(f"Recycle split set to {recycle_split_ratio*100:.1f}%")
    else:
        # Initialize final concentrate product
        propagate_state(arc=m.fs.final_conc_arc)
        m.fs.concentrate_product.initialize()
    
    # Solve the full model
    print("\nSolving complete model...")
    if has_recycle:
        # Solve with recycle - may need iterations
        print("Solving with recycle flow...")
        solver.options['max_iter'] = 200
        solver.options['tol'] = 1e-6
    
    results = solver.solve(m, tee=True)
    
    if results.solver.termination_condition == TerminationCondition.optimal:
        print("\nSolution found!")
        
        # Report key results
        if has_recycle:
            recycle_flow = value(m.fs.recycle_split.recycle.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                               m.fs.recycle_split.recycle.flow_mass_phase_comp[0, 'Liq', 'TDS'])
            disposal_flow = value(m.fs.disposal_product.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'] + 
                                m.fs.disposal_product.inlet.flow_mass_phase_comp[0, 'Liq', 'TDS'])
            print(f"\nRecycle flow: {recycle_flow*3.6:.1f} m3/h")
            print(f"Disposal flow: {disposal_flow*3.6:.1f} m3/h")
    
    return results

In [ ]:
# Build model
try:
    print("Building RO model with recycle support...")
    m = build_ro_model_with_recycle(configuration, feed_salinity_ppm, feed_temperature_c, 
                                    membrane_type, has_recycle)
    
    print("\nInitializing and solving model...")
    solve_results = initialize_and_solve_with_recycle(m, configuration, has_recycle, optimize_pumps)
    
    if solve_results.solver.termination_condition == TerminationCondition.optimal:
        print("\nExtracting results...")
        results = extract_results_with_recycle(m, configuration, has_recycle)
        results["message"] = "Simulation completed successfully"
        if has_recycle:
            results["message"] += " with recycle"
    else:
        results = {
            "status": "error",
            "message": f"Solver failed: {solve_results.solver.termination_condition}",
            "performance": {},
            "economics": {},
            "stage_results": [],
            "mass_balance": {},
            "recycle_info": None
        }
        
except Exception as e:
    import traceback
    results = {
        "status": "error",
        "message": f"Simulation error: {str(e)}",
        "traceback": traceback.format_exc(),
        "performance": {},
        "economics": {},
        "stage_results": [],
        "mass_balance": {},
        "recycle_info": None
    }

print("\nSimulation complete.")

## Display Results

In [ ]:
# Display results summary
import json
print("\n" + "="*60)
print("SIMULATION RESULTS")
print("="*60)

if results.get("status") == "success":
    print(f"\nOverall Performance:")
    print(f"  Total Recovery (from fresh feed): {results['performance']['total_recovery']*100:.1f}%")
    if has_recycle:
        print(f"  System Recovery (from effective feed): {results['performance']['system_recovery']*100:.1f}%")
    print(f"  Specific Energy: {results['economics']['specific_energy_kwh_m3']:.2f} kWh/m3")
    print(f"  Total Power: {results['economics']['total_power_kw']:.1f} kW")
    
    if has_recycle and results.get('recycle_info'):
        print(f"\nRecycle Information:")
        print(f"  Recycle Flow: {results['recycle_info']['recycle_flow_m3h']:.1f} m3/h")
        print(f"  Disposal Flow: {results['recycle_info']['disposal_flow_m3h']:.1f} m3/h")
        print(f"  Recycle Ratio: {results['recycle_info']['recycle_ratio']*100:.1f}%")
        print(f"  Effective Feed TDS: {results['recycle_info']['effective_feed_tds_ppm']:.0f} ppm")
    
    print(f"\nStage Results:")
    for stage in results['stage_results']:
        print(f"  Stage {stage['stage_number']}:")
        print(f"    Pressure: {stage['feed_pressure_bar']:.1f} bar")
        print(f"    Recovery: {stage['water_recovery']*100:.1f}%")
        print(f"    Permeate: {stage['permeate_flow_m3h']:.1f} m3/h @ {stage['permeate_tds_ppm']:.0f} ppm")

# Full results as JSON
print("\n" + "="*60)
print("FULL RESULTS (JSON):")
print("="*60)
print(json.dumps(results, indent=2, default=str))

In [ ]:
# Results cell - tagged for papermill to extract
results